In [ ]:
conda install -c conda-forge cudatoolkit==11.7.0
conda install -c conda-forge cudnn
conda install -c conda-forge cudatoolkit-dev
conda install -c conda-forge pytorch==1.13.1
conda install -c conda-forge nvidia-apex
git clone https://github.com/hpcaitech/ColossalAI.git
cd ColossalAI/applications/Chat
CUDA_EXT=1 pip install .
git clone https://github.com/Chenrj233/ParlAI.git
cd ParlAI
python setup.py install

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain './checkpoint/llama_7B' \
    --model 'llama' \
    --strategy colossalai_zero2 \
    --log_interval 128 \
    --save_path './checkpoint_llama/step1/epoch1' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 1 \
    --lora_rank 16 \
    --batch_size 4 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain './llama-2-7b-chat-hf/' \
    --model 'llama' \
    --strategy colossalai_zero2_cpu \
    --log_interval 128 \
    --save_path './checkpoint_llama2/step1/epoch1-1e-5' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 1e-5 \
    --max_epochs 1 \
    --batch_size 1 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain 'facebook/opt-2.7b' \
    --model 'opt' \
    --strategy colossalai_zero2 \
    --log_interval 128 \
    --save_path './checkpoint_opt2.7B/step1/epoch1-3e-5' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 3e-5 \
    --max_epochs 1 \
    --batch_size 2 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain './llama-2-7b-chat-hf/' \
    --model 'llama' \
    --strategy colossalai_zero2_cpu \
    --log_interval 128 \
    --save_path './checkpoint_llama2_twodata/step1/epoch1_dstc_2e5' \
    --dataset 'dstc' \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 1 \
    --batch_size 1 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain './checkpoint_llama2_twodata/step1/epoch1_dstc_8e6' \
    --model 'llama' \
    --strategy colossalai_zero2_cpu \
    --log_interval 128 \
    --save_path './checkpoint_llama2_twodata/step1/epoch1_persona_8e6_8e6re' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 8e-6 \
    --max_epochs 1 \
    --batch_size 1 \
    --max_datasets_size 128 \
    --revised

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain './checkpoint_llama2_twodata/step1/epoch1_persona_8e6_8e6re' \
    --model 'llama' \
    --strategy colossalai_zero2_cpu \
    --log_interval 128 \
    --save_path './checkpoint_llama2_twodata/step1/epoch1_persona_8e6_8e6re_1e5' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 1e-5 \
    --max_epochs 1 \
    --batch_size 1 \
    --max_datasets_size 128 \
    --revised

In [ ]:
torchrun --standalone --nproc_per_node=3 train_sft.py \
    --pretrain 'facebook/opt-iml-max-1.3b' \
    --model 'opt' \
    --strategy colossalai_zero2 \
    --log_interval 128 \
    --save_path './checkpoint_optmax_twodata/step1/epoch1_dstc_2e6' \
    --dataset 'dstc' \
    --accumulation_steps 8 \
    --lr 2e-6 \
    --max_epochs 1 \
    --batch_size 8 \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=1 train_sft.py \
    --pretrain './checkpoint_optmax_twodata/step1/epoch1_dstc_2e5' \
    --model 'opt' \
    --strategy colossalai_zero2 \
    --log_interval 128 \
    --save_path './checkpoint_optmax_twodata/step1/epoch1_persona_2e6_2e5' \
    --dataset 'PersonaChat' \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 1 \
    --batch_size 8 \
    --max_datasets_size 128 \
    --revised true

In [ ]:
torchrun --standalone --nproc_per_node=3 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch3' \
    --dataset 'Dahoas/rm-static' \
    --save_path './checkpoint_opt/step2/epoch3/rmstatic.pt' \
    --max_epochs 3 \
    --batch_size 16 \
    --loss_fn 'log_exp' \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_reward_model.py \
    --strategy colossalai_gemini \
    --model 'opt' \
    --pretrain './checkpoint_llama/step1/epoch1_cpu' \
    --dataset 'PersonaChat' \
    --save_path './checkpoint_llama/step2/epoch1_gemini/rmpersona.pt' \
    --max_epochs 1 \
    --batch_size 2 \
    --loss_fn 'log_exp' \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=3 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt1.3Biml/step1/epoch5' \
    --dataset 'PersonaChat' \
    --save_path './checkpoint_opt1.3Biml/step2/epoch5/rmpersona.pt' \
    --max_epochs 5 \
    --batch_size 8 \
    --loss_fn 'log_exp' \
    --max_datasets_size 128

In [ ]:
torchrun --standalone --nproc_per_node=1 train_prompts.py \
    --prompt_dataset 'PersonaChat' \
    --pretrain_dataset 'PersonaChat' \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch1' \
    --rm_model 'opt' \
    --rm_pretrain 'facebook/opt-2.7b' \
    --rm_path './checkpoint_opt/step2/epoch1/rmstatic.pt' \
    --save_path './checkpoint_opt/step3/epoch1/acstatic.pt' \
    --num_episodes 10 \
    --max_epochs 1 \
    --train_batch_size 8 \
    --ptx_batch_size 1 \
    --experience_batch_size 8\
    --lora_rank 16 \
    --max_input_len 511 \
    --max_seq_len 512 \
    --tokenizer 'facebook/opt-350m' \
    --max_datasets_size 16

In [ ]:
torchrun --standalone --nproc_per_node=3 train_prompts.py \
    --prompt_dataset 'PersonaChat' \
    --pretrain_dataset 'PersonaCha1t' \
    --strategy colossalai_zero2 \
    --model 'llama' \
    --pretrain './checkpoint_llama/step1/epoch1_cpu' \
    --rm_model 'llama' \
    --rm_pretrain './checkpoint/llama_7B' \
    --rm_path './checkpoint_llama/step2/epoch1_gemini/rmpersona.pt' \
    --save_path './checkpoint_llama/step3/epoch1/rmpersona.pt' \
    --num_episodes 10 \
    --max_epochs 1 \
    --train_batch_size 1 \
    --ptx_batch_size 1 \
    --experience_batch_size 1\
    --lora_rank 16 \
    --max_input_len 511 \
    --max_seq_len 512 \
    --tokenizer './checkpoint_opt/step1/epoch3' \
    --max_datasets_size 128


In [ ]:
torchrun --standalone --nproc_per_node=3 train_prompts.py \
    --prompt_dataset 'PersonaChat' \
    --pretrain_dataset 'PersonaChat' \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt1.3Biml/step1/epoch5' \
    --rm_model 'opt' \
    --rm_pretrain './checkpoint_opt1.3Biml/step1/epoch5' \
    --rm_path './checkpoint_opt1.3Biml/step2/epoch5/rmpersona.pt' \
    --save_path './checkpoint_opt1.3Biml/step3/epoch51/rmpersona.pt' \
    --num_episodes 10 \
    --max_epochs 1 \
    --train_batch_size 1 \
    --ptx_batch_size 1 \
    --experience_batch_size 8\
    --max_input_len 511 \
    --max_seq_len 512 \
    --tokenizer './checkpoint_opt1.3Biml/step1/epoch5' \
    --max_datasets_size 128

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint 'persona_original' \
    --eval_type f1 \
    --beam 2 \
    --max_history 10 \
    --revised

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint './checkpoint_llama2/step1/epoch1-1e-5' \
    --eval_type f1 \
    --beam 2 \
    --max_history 10 

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint './checkpoint_llama2_twodata/step1/epoch1_persona_8e6_1e5' \
    --eval_type f1 \
    --beam 2 \
    --max_history 10 

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint './checkpoint_optmax_twodata/step1/epoch1_persona_2e5_2e5' \
    --eval_type f1 \
    --beam 2 \
    --max_history 10 \
    --revised

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint 'persona_original' \
    --eval_type hits@1 

In [ ]:
python evaluation_PersonaChat.py \
    --model_checkpoint './checkpoint_llama2_twodata/step1/epoch1_persona_2e6_2e5' \
    --eval_type hits@1 

In [ ]:
python convert_llama_weights_to_hf.py \
    --input_dir './llama' \
    --model_size '7Bf' \
    --output_dir './llama-2-7b-chat-hf/'

In [ ]:
torchrun --standalone --nproc_per_node=1 inference.py \
    --model 'opt' \
    --pretrain 'facebook/opt-2.7b' \
    --model_path './checkpoint_opt/step3/epoch1/acstatic.pt' \
    --max_length 128 \
    --tokenizer 'facebook/opt-350m'   

In [ ]:
torchrun --standalone --nproc_per_node=1 inference.py \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch1' \
    --max_length 512 \
    --tokenizer 'facebook/opt-350m'   

In [ ]:
tensor([[-14.5536],
        [-12.7583],
        [-16.2968],
        [ -9.1099],
        [-15.5924],
        [-15.5844],
        [ -5.8899],
        [-16.1088],
        [-13.6400],
        [-11.5683],
        [ -3.5833],
        [-12.0773],
        [ -3.8775],
        [-15.0167],
        [-12.2481],
        [-16.2931],
        [-11.0122],
        [-16.8578],
        [-16.3747],
        [ -4.3238]], device='cuda:0')
20
1

In [ ]:
tensor([[-5.1600,  5.5538, 11.0944,  ...,  4.2302,  0.3960, 25.2892],
        [-6.1038, -0.8699,  4.2030,  ...,  1.7245, -1.8477,  2.9103],
        [-7.1050, -3.8702,  2.4604,  ..., -3.3658, -0.8522,  0.0561],
        ...,
        [ 0.3380,  2.2908, 19.6358,  ...,  3.5377, -3.0058,  5.2599],
        [-0.3818,  0.8998, 20.4717,  ...,  0.7886, -1.9366,  4.5995],
        [-0.9332,  0.0713,  3.8940,  ..., 28.3136, -3.9910,  2.1242]],
       device='cuda:0')
49
32006

In [ ]:
epoch5
1. 0.03676635443136688
2. 0.02773737145198687

In [ ]:
docker run -it --gpus all -p 8899:8888 -e JUPYTER_ENABLE_LAB=yes -v /mnt/userhome/sun/jupyter:/home/jovyan/work persona_sun_v2 &